In [2]:
import requests
import csv
import re
from bs4 import BeautifulSoup

def extract_tasting_notes_from_blend(body_html):
    """
    Extracts tasting/flavour notes for blend products.
    
    First, it looks for an <em> tag and returns its text if found.
    If no <em> tag is present, it falls back to checking the first <p> tag 
    for a vertical bar ("|") and returning the text after it.
    """
    soup = BeautifulSoup(body_html, 'html.parser')
    # Try to find all <em> tags
    em_tag = soup.find('em')
    if em_tag:
        return em_tag.get_text().strip()
    
    # Fallback: look in the first <p> tag for a vertical bar.
    first_p = soup.find('p')
    if first_p:
        text = first_p.get_text(separator=" ", strip=True)
        if "|" in text:
            parts = text.split("|")
            if len(parts) > 1:
                return parts[1].strip()
    return ""

def extract_cupping_notes(body_html):
    """
    Extracts cupping notes for single origin products by checking for the first occurrence 
    of either an <em> or <i> tag.
    """
    soup = BeautifulSoup(body_html, 'html.parser')
    note_tag = soup.find(['em', 'i'])
    if note_tag:
        return note_tag.get_text().strip()
    return ""

def extract_table_info(body_html):
    """
    Extracts table information for Region, Varietal, Process, and Altitude.
    
    The function finds the first <table> element and processes its second <td> cell.
    
    - For many products, the cell splits into four parts:
         [Region, Varietal, Process, Altitude]
    - For products like "Midnight Run" where the table splits into five parts,
         we assume the parts represent [Country, Farms, Varietals, Process, Altitude]
         and we map as follows:
             Region ← part 0 (Country)
             Varietal ← part 2 (Varietals)
             Process ← part 3
             Altitude ← part 4
    """
    result = {"Region": "", "Varietal": "", "Process": "", "Altitude": ""}
    soup = BeautifulSoup(body_html, 'html.parser')
    table = soup.find('table')
    if table:
        tds = table.find_all('td')
        if len(tds) >= 2:
            # Use "|" as a temporary separator for <br> tags.
            cell_text = tds[1].get_text(separator="|")
            parts = [part.strip() for part in cell_text.split("|") if part.strip() != ""]
            # Remove any leading colon from each part.
            parts = [part.lstrip(":").strip() for part in parts]
            if len(parts) == 5:
                result["Region"] = parts[0]
                result["Varietal"] = parts[2]
                result["Process"] = parts[3]
                result["Altitude"] = parts[4]
            elif len(parts) >= 4:
                result["Region"] = parts[0]
                result["Varietal"] = parts[1]
                result["Process"] = parts[2]
                result["Altitude"] = parts[3]
    return result

def extract_roast_type(body_html):
    """
    Extracts roast type from the first <p> tag in the body_html.
    It looks for a vertical bar ("|") and takes the text after it,
    then normalizes the value to "espresso", "filter", or "omni".
    """
    soup = BeautifulSoup(body_html, 'html.parser')
    first_p = soup.find('p')
    if first_p:
        text = first_p.get_text(separator=" ", strip=True)
        if "|" in text:
            parts = text.split("|")
            roast = parts[1].strip().lower()
            if "espresso" in roast:
                return "espresso"
            elif "filter" in roast:
                return "filter"
            elif "omni" in roast:
                return "omni"
    return ""

def process_products(url, csv_filename):
    # Fetch the JSON from the URL.
    response = requests.get(url)
    if response.status_code != 200:
        print("Error fetching products:", response.status_code)
        return
    data = response.json()
    products = data.get("products", [])

    # Define CSV columns.
    fieldnames = [
        "Title",
        "Primary Image URL",
        "Availability",
        "Roast Type",
        "Flavour Notes",
        "Region",
        "Varietal",
        "Process",
        "Altitude",
        "Vendor",
        "Product Type",
        "Variant Part 1",
        "Variant Part 2",
        "Variant Part 3",
        "Variant Price",
        "Variant Availability"
    ]

    with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for product in products:
            title = product.get("title", "")
            vendor = product.get("vendor", "")
            product_type = product.get("product_type", "")
            images = product.get("images", [])
            primary_image_url = images[0]["src"] if images else ""
            product_availability = any(variant.get("available", False) for variant in product.get("variants", []))
            availability = "Available" if product_availability else "Not Available"
            body_html = product.get("body_html", "")

            # Extract roast type.
            roast_type = extract_roast_type(body_html)
            
            # Choose note extraction based on product type.
            if "blend" in product_type.lower():
                notes = extract_tasting_notes_from_blend(body_html)
            elif "single origin" in product_type.lower():
                notes = extract_cupping_notes(body_html)
            else:
                notes = ""

            # Extract table info.
            table_info = extract_table_info(body_html)
            region = table_info.get("Region", "")
            varietal = table_info.get("Varietal", "")
            process_field = table_info.get("Process", "")
            altitude = table_info.get("Altitude", "")

            # Loop through each variant.
            for variant in product.get("variants", []):
                variant_name = variant.get("title", "")
                # Only process variants that include "Whole Bean" (case-insensitive)
                # and skip variants containing "(Wholesale)".
                if "whole bean" not in variant_name.lower():
                    continue
                if "(wholesale)" in variant_name.lower():
                    continue

                parts = [p.strip() for p in variant_name.split(" / ")]
                if len(parts) < 3:
                    parts += [""] * (3 - len(parts))
                variant_part1, variant_part2, variant_part3 = parts[:3]

                variant_price = variant.get("price", "")
                variant_availability = "Available" if variant.get("available", False) else "Not Available"

                row = {
                    "Title": title,
                    "Primary Image URL": primary_image_url,
                    "Availability": availability,
                    "Roast Type": roast_type,
                    "Flavour Notes": notes,
                    "Region": region,
                    "Varietal": varietal,
                    "Process": process_field,
                    "Altitude": altitude,
                    "Vendor": vendor,
                    "Product Type": product_type,
                    "Variant Part 1": variant_part1,
                    "Variant Part 2": variant_part2,
                    "Variant Part 3": variant_part3,
                    "Variant Price": variant_price,
                    "Variant Availability": variant_availability
                }
                writer.writerow(row)

    print(f"CSV file '{csv_filename}' created with {len(products)} products processed.")

if __name__ == "__main__":
    # URL for the Dutch Colony products JSON.
    url = "https://www.dutchcolony.sg/collections/coffee-2/products.json"
    csv_filename = "combined_extracted_product_details.csv"
    process_products(url, csv_filename)

CSV file 'combined_extracted_product_details.csv' created with 19 products processed.
